In [2]:
from torch_geometric.datasets import TUDataset
import sys
sys.path.append('..')
from dhg import Hypergraph
from hypgs.models.hsn_pyg import HSN
from hypgs.utils.data import HGDataset
from torch_geometric.loader import DataLoader
from torch_geometric.data import Dataset
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
from torch.utils.data import random_split
from pytorch_lightning.loggers import TensorBoardLogger


In [3]:
# original_dataset = TUDataset(root='../data/', name="MUTAG", use_node_attr=True)
original_dataset = TUDataset(root='../data/', name="ENZYMES", use_node_attr=True)
to_hg_func = lambda g: Hypergraph.from_graph_kHop(g, 1)
dataset = HGDataset(original_dataset, to_hg_func)
in_channels = dataset.num_node_features
out_channels = dataset.num_classes

In [4]:
# Assuming 'dataset' is your PyTorch Dataset
dataset_size = len(dataset)
train_size = int(0.7 * dataset_size)
val_size = int(0.15 * dataset_size)
test_size = dataset_size - (train_size + val_size)

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [4]:
model = HSN(in_channels=in_channels,
              hidden_channels=64,
              out_channels = out_channels,
              trainable_laziness = False,
              trainable_scales = False, 
              activation = 'modulus', 
              fixed_weights=True, 
              layout=['hsm', 'hsm'], 
              normalize='right', 
              pooling='max',
              task='classification',
        )

In [5]:
# Early stopping callback based on validation loss
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, verbose=True, mode='min')
logger = TensorBoardLogger("lightning_logs", name="my_model")

trainer = pl.Trainer(
    max_epochs=100,  # Adjust as necessary
    logger=logger,
    callbacks=[early_stopping_callback],
)

# Assuming you have defined train_loader and val_loader
trainer.fit(model, train_loader, val_loader)  # Corrected argument names
# Test the model
test_result = trainer.test(model, test_loader)

/home/sumry2023_cqx3/.conda/envs/hyper/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/sumry2023_cqx3/.conda/envs/hyper/lib/python3.1 ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: lightning_logs/my_model
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type               | Params
---------------------------------------------------
0 | layers      | ModuleList         | 204   
1 | batch_norm  | BatchNorm          | 1.5 K 
2 | fc1         | Linear             | 286 K 
3 | fc2         | Linear             | 2.3 K 
4 | relu        | ReLU               | 0     
5 | batch_norm1 | BatchNorm1d        | 756   
6 | mlp         | Sequential

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/pyg/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.
/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/pyg/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 1128. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/pyg/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 1047. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/pyg/lib/p

Training: |          | 0/? [00:00<?, ?it/s]

/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/pyg/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 1041. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/pyg/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 1097. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/pyg/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 1013. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/pyg/lib/python3.9/site-packages/pytorch_lightning/utilitie

...

Validation: |          | 0/? [00:00<?, ?it/s]

Monitored metric val_loss did not improve in the last 10 records. Best score: 1.171. Signaling Trainer to stop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/pyg/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/pyg/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 1237. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/pyg/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 898. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     test_acc_epoch          0.60150545835495
     test_auc_epoch         0.8450121283531189
     test_loss_epoch        1.7098710536956787
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [6]:
test_result

[{'test_loss_epoch': 1.7098710536956787,
  'test_acc_epoch': 0.60150545835495,
  'test_auc_epoch': 0.8450121283531189}]

In [7]:
# # to see the logs run
# !tensorboard --logdir lightning_logs

TensorFlow installation not found - running with reduced feature set.
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.15.1 at http://localhost:6006/ (Press CTRL+C to quit)
^C
